Needleman-wunsch algorithm

Objective: To achieve global pairwaise sequence alignment by Needleman-Wunsch Algorithm

In [11]:
def get_sequence_from_fasta(content):
    lines = content.strip().splitlines()
    # Check if the first line is a FASTA header (starts with ">")
    if lines[0].startswith(">"):
        sequence = ''.join(lines[1:])  # Concatenate lines after the header
    else:
        sequence = ''.join(lines)  # Treat all lines as sequence if no header
    return sequence

In [12]:
#Initializing scoring matrix and traceback matrix
def initialize_matrices(l1, l2, gap):
    score = [[0 for _ in range(l1 + 1)] for _ in range(l2 + 1)]  # Initialize score matrix
    tb = [[[0, 0, 0] for _ in range(l1 + 1)] for _ in range(l2 + 1)]  # Initialize traceback matrix
    
    for j in range(1, l2 + 1):
        score[j][0] = score[j - 1][0] + gap  # Fill the first column with gap penalties
        tb[j][0][2] = 1  # Mark traceback for vertical gaps
        
    for i in range(1, l1 + 1):
        score[0][i] = score[0][i - 1] + gap  # Fill the first row with gap penalties
        tb[0][i][1] = 1  # Mark traceback for horizontal gaps
        
    return score, tb

In [13]:
#filling the respective values in the scoring matrix table
def fill_matrices(l1, l2, string1, string2, match, mismatch, gap, score, tb):
    for j in range(1, l2 + 1):
        for i in range(1, l1 + 1):
            m = match if string1[i - 1] == string2[j - 1] else mismatch  # Match or mismatch score
            s0 = score[j - 1][i - 1] + m  # Diagonal move
            s1 = score[j][i - 1] + gap  # Horizontal move
            s2 = score[j - 1][i] + gap  # Vertical move
            score[j][i] = max(s0, s1, s2)  # Choose the highest score
            
            tb[j][i] = [s == score[j][i] for s in (s0, s1, s2)]  # Update traceback

In [14]:
#creating a function for the traceback mamtrix to find the traceback
def traceback(l1, l2, string1, string2, tb):
    gene1 = " " + string1  # Add a leading space for easier indexing
    gene2 = " " + string2
    
    as1, as2, bar = "", "", ""  # Alignment strings
    mc, x, y = 0, l1, l2  # Match count, starting coordinates
    
    while x > 0 or y > 0:
        if tb[y][x][0]:  # Diagonal move
            as1 += gene1[x]
            as2 += gene2[y]
            bar += '|' if gene1[x] == gene2[y] else ' '  # Match or mismatch
            mc += gene1[x] == gene2[y]
            x, y = x - 1, y - 1
        elif tb[y][x][1]:  # Horizontal move
            as1 += gene1[x]
            as2 += "_"
            bar += " "
            x -= 1
        elif tb[y][x][2]:  # Vertical move
            as1 += "_"
            as2 += gene2[y]
            bar += " "
            y -= 1
        else:
            break
    
    return as1[::-1], as2[::-1], bar[::-1], mc

In [15]:
def print_alignment(as1, as2, bar):
    k = 0
    while k * 100 < len(as1):
        print(as1[100 * k: 100 * (k + 1)])  # Print chunks of 100 characters
        print(bar[100 * k: 100 * (k + 1)])
        print(as2[100 * k: 100 * (k + 1)])
        print("\n")
        k += 1

In [ ]:
if __name__ == "__main__":
    gap = int(input("Enter the gap penalty: "))
    match = int(input("Enter the match score: "))
    mismatch = int(input("Enter the mismatch penalty: "))
    
    print("\nEnter the sequences. Copy & Paste the FASTA file content :- ")
    
    content1 = input("\nEnter the first sequence: ")
    string1 = get_sequence_from_fasta(content1)
    
    content2 = input("Enter the second sequence: ")
    string2 = get_sequence_from_fasta(content2)
    
    # Check if sequences are empty
    if not string1 or not string2:
        print("Error: One or both sequences are empty. Please input valid sequences.")
        exit()
    
    print("\nString1 is\n", string1)
    print("\nString2 is\n", string2)
    
    l1, l2 = len(string1), len(string2)
    print("\nThe lengths of the sequences are", l1, l2, end='.')
    print("\n\n")
    
    score, tb = initialize_matrices(l1, l2, gap)
    fill_matrices(l1, l2, string1, string2, match, mismatch, gap, score, tb)
    
    as1, as2, bar, mc = traceback(l1, l2, string1, string2, tb)
    print_alignment(as1, as2, bar)
    
    if l2 > l1:
        l1 = l2
    print("The number of matches is", mc)
    
    # Check for zero-length sequences to avoid division by zero
    if l1 == 0:
        print("Error: Sequence length is zero. Cannot compute percent similarity.")
    else:
        print("The percent similarity is", mc * 100 / l1)